# ETL DEL DATASET PRINCIPAL: MOVIES

In [5]:
#Es necesario importar las depencendias necesarias
import pandas as pd
import numpy as np
import json
from datetime import date, datetime

In [6]:
# Importamos el dataset de trabajo
df = pd.read_csv('./dataset/movies_dataset.csv', delimiter=',')

# Se elimina las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage
df = df.drop(columns = ['video','imdb_id','adult','original_title','poster_path','homepage']) 
# df.head()
print(" El dataset tiene", df.shape[0],"filas. ")

 El dataset tiene 45466 filas. 


C:\Users\HP\AppData\Local\Temp\ipykernel_15452\1522230964.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./dataset/movies_dataset.csv', delimiter=',')


In [7]:
# Se accede a la columna "belongs_to_collection" y se visualiza los primeros 10 valores
df["belongs_to_collection"].head(10) 

0    {'id': 10194, 'name': 'Toy Story Collection', ...
1                                                  NaN
2    {'id': 119050, 'name': 'Grumpy Old Men Collect...
3                                                  NaN
4    {'id': 96871, 'name': 'Father of the Bride Col...
5                                                  NaN
6                                                  NaN
7                                                  NaN
8                                                  NaN
9    {'id': 645, 'name': 'James Bond Collection', '...
Name: belongs_to_collection, dtype: object

In [8]:
# se corrige los campos float
df['belongs_to_collection'].replace('0,065736',None, inplace = True)
df['belongs_to_collection'].replace('1,931659',None, inplace = True)
df['belongs_to_collection'].replace('2,185485',None, inplace = True)

# Se reemplaza el formato de comillas en la columna 'belongs_to_collection'
df['belongs_to_collection'].replace("'",'"', inplace = True) 
# se pasa a string
df['belongs_to_collection'] = df['belongs_to_collection'].astype(str)
subdicc = dict()

for i in range(len(df)):
    franquicia = df.iloc[i]['belongs_to_collection']
    if pd.notna(franquicia):  # Se verifica si no es un valor nulo o NaN
    
        try:
            subdicc = eval(franquicia)  # Usar eval para convertir la cadena en un diccionario
            if isinstance(subdicc, dict):
                #print(subdicc.get('name'))
                df.at[i, 'franquicia'] = subdicc.get('name')
            else:
                #print("El valor no es un diccionario válido.")
                df.at[i, 'franquicia'] = None
        except:
            df.at[i, 'franquicia'] = None
    else:
        df.at[i, 'franquicia'] = None

df['franquicia']

0                  Toy Story Collection
1                                  None
2             Grumpy Old Men Collection
3                                  None
4        Father of the Bride Collection
                      ...              
45461                              None
45462                              None
45463                              None
45464                              None
45465                              None
Name: franquicia, Length: 45466, dtype: object

In [9]:
print(df)

                                   belongs_to_collection    budget  \
0      {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1                                                    nan  65000000   
2      {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3                                                    nan  16000000   
4      {'id': 96871, 'name': 'Father of the Bride Col...         0   
...                                                  ...       ...   
45461                                                nan         0   
45462                                                nan         0   
45463                                                nan         0   
45464                                                nan         0   
45465                                                nan         0   

                                                  genres      id  \
0      [{'id': 16, 'name': 'Animation'}, {'id': 35, '...     862   
1      [{'id': 12, 'nam

In [11]:
# production_companies , listado de productoras

lista_productoras = []

for i in range(len(df)):
    print(df.iloc[i]['production_companies'])
    lista_productoras = df.iloc[i]['production_companies']
    
    try:
        lista_productoras    = json.loads(lista_productoras)
    except json.JSONDecodeError:
        lista_productoras = None
    # Se pasa la cadena JSON a una lista de diccionarios
   
    if isinstance(lista_productoras, list) and pd.notna(lista_productoras):  
        # Se recorre los diccionarios
        for valor in lista_productoras:
            try:
                # Se carfa la cadena JSON como una lista de diccionarios
                lista_diccionarios = json.loads(valor.replace("'", "\""))
                productora_nombre  = ''
                list_productora    = []
                # Recorrer los diccionarios y obtener los valores de la clave 'name'
                for diccionario in lista_diccionarios:
                    productora_nombre = diccionario.get('name')
                    if productora_nombre:
                        list_productora.append(productora_nombre)
               
                df.at[i, 'productoras'] = list_productora
            
            except json.JSONDecodeError:
                df.at[i, 'productoras'] = None            

[{'name': 'Pixar Animation Studios', 'id': 3}]
[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Interscope Communications', 'id': 10201}]
[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancaster Gate', 'id': 19464}]
[{'name': 'Twentieth Century Fox Film Corporation', 'id': 306}]
[{'name': 'Sandollar Productions', 'id': 5842}, {'name': 'Touchstone Pictures', 'id': 9195}]
[{'name': 'Regency Enterprises', 'id': 508}, {'name': 'Forward Pass', 'id': 675}, {'name': 'Warner Bros.', 'id': 6194}]
[{'name': 'Paramount Pictures', 'id': 4}, {'name': 'Scott Rudin Productions', 'id': 258}, {'name': 'Mirage Enterprises', 'id': 932}, {'name': 'Sandollar Productions', 'id': 5842}, {'name': 'Constellation Entertainment', 'id': 14941}, {'name': 'Worldwide', 'id': 55873}, {'name': 'Mont Blanc Entertainment GmbH', 'id': 58079}]
[{'name': 'Walt Disney Pictures', 'id': 2}]
[{'name': 'Universal Pictures', 'id': 33}, {'name': 'Imperial Entertainment', 'id': 21437}, {'

C:\Users\HP\AppData\Local\Temp\ipykernel_15452\699698970.py:15: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if isinstance(lista_productoras, list) and pd.notna(lista_productoras):


[{'name': 'Paramount Pictures', 'id': 4}]
[{'name': 'Geffen Pictures', 'id': 360}]
[{'name': 'Merchant Ivory Productions', 'id': 2370}, {'name': 'Touchstone Pictures', 'id': 9195}]
[{'name': 'Caravan Pictures', 'id': 175}, {'name': 'Touchstone Pictures', 'id': 9195}]
[{'name': 'Universal Pictures', 'id': 33}, {'name': 'Northern Lights Entertainment', 'id': 8816}]
[{'name': 'Fountainbridge Films', 'id': 414}, {'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lee Rich Productions', 'id': 36854}]
[{'name': 'Walt Disney Pictures', 'id': 2}, {'name': 'Trimark Pictures', 'id': 4063}]
[{'name': 'Twentieth Century Fox Film Corporation', 'id': 306}]
[{'name': 'Lucasfilm', 'id': 1}, {'name': 'Twentieth Century Fox Film Corporation', 'id': 306}]
[{'name': 'Columbia Pictures', 'id': 5}]
[{'name': 'Warner Bros.', 'id': 6194}]
[{'name': 'Channel Four Films', 'id': 181}, {'name': 'Parallax Pictures', 'id': 983}]
[{'name': 'MK2 Productions', 'id': 9209}]
[{'name': 'Aviacsa', 'id': 84803}, {'name': 'Cine

TypeError: the JSON object must be str, bytes or bytearray, not float

In [12]:
df['productoras'] 

KeyError: 'productoras'

In [48]:
# Se averigua los nulos, con enfoque en los campos revenue y budget .
df.isnull().sum()

belongs_to_collection        0
budget                       0
genres                       0
id                           0
original_language           11
overview                   941
popularity                   2
production_companies         0
production_countries         0
release_date                 3
revenue                      0
runtime                    249
spoken_languages             3
status                      83
tagline                  24981
title                        3
vote_average                 3
vote_count                   3
franquicia               40891
release_year                 3
retorno                      0
productoras              45378
dtype: int64

In [11]:
# COLUMNA revenue
# Imputamos con el valor 0 los campos nulos de la con una revenue - recaudacion de la pelicula

df['revenue'].fillna(0, inplace=True)

df.isnull().sum()

belongs_to_collection        0
budget                       0
genres                       0
id                           0
original_language           11
overview                   954
popularity                   5
production_companies         3
production_countries         3
release_date                87
revenue                      0
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
vote_average                 6
vote_count                   6
franquicia               40975
dtype: int64

In [12]:
# Los valores nulos del campo release date (fecha de estreno) deben eliminarse. 
df.dropna(subset='release_date',inplace=True )
# Adicionalmente se hallaron valores 1,12 y 22 en release_date que fueron remplazados por None,
# para luego pasar esta columna a datetime.
df['release_date'].replace('1',None, inplace = True)
df['release_date'].replace('12',None, inplace = True)
df['release_date'].replace('22',None, inplace = True)

df.isnull().sum()


belongs_to_collection        0
budget                       0
genres                       0
id                           0
original_language           11
overview                   941
popularity                   2
production_companies         0
production_countries         0
release_date                 3
revenue                      0
runtime                    249
spoken_languages             3
status                      83
tagline                  24981
title                        3
vote_average                 3
vote_count                   3
franquicia               40891
dtype: int64

In [13]:
# De haber fechas, deberán tener el formato AAAA-mm-dd, 
# además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.
df['release_date']          = pd.to_datetime(df['release_date'])
df['release_year']          = df['release_date'].dt.year 

df.head()


,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,franquicia,release_year
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,1995.0
1,nan,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,None,1995.0
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,1995.0
3,nan,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,None,1995.0
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,Father of the Bride Collection,1995.0


In [15]:
# budge es el presupuesto de la pelicula remplazar los valores incorrectos por cero y luego pasat a float
df['budget'].replace('/ff9qCepilowshEtG2GYWwzt2bs4.jpg', 0, inplace = True)
df['budget'].replace('/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg', 0, inplace = True)
df['budget'].replace('/zaSf5OG7V8X8gqFvly88zDdRm46.jpg', 0, inplace = True)

In [18]:
""" Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, 
    dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, 
    deberá tomar el valor 0.
"""
# Se convierte los campos 'revenue' y 'budget' a valores numéricos (float)
df['revenue'] = df['revenue'].astype(float)
df['budget'] = df['budget'].astype(float)

# Se calcula el retorno de inversión y reemplaza los valores faltantes con 0
df['retorno'] = np.where((df['revenue'] > 0) & (df['budget'] > 0), df['revenue'] / df['budget'], 0)

df['retorno'] 


0        12.451801
1         4.043035
2         0.000000
3         5.090760
4         0.000000
           ...    
45460     0.000000
45462     0.000000
45463     0.000000
45464     0.000000
45465     0.000000
Name: retorno, Length: 45379, dtype: float64

In [19]:
df.to_csv('../movies/dataset/movies_limpio.csv', index=False)